In [5]:
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm

In [17]:
segnet_path = '/kaggle/input/segnet_model/tensorflow2/default/1/segnet_model.keras'
unet_path = '/kaggle/input/unet_model_1/tensorflow2/default/1/unet_model20epoches.keras'

# Загрузка обученной модели
segnet_model = tf.keras.models.load_model(segnet_path)
unet_model = tf.keras.models.load_model(unet_path)

In [18]:
def process_frame_one_model(frame, it, model):
    """Обрабатывает один кадр, выделяя человека и удаляя фон."""
    original_frame = frame.copy()
    h, w, _ = frame.shape

    # Подготовка кадра для модели
    frame_resized = cv2.resize(frame, (256, 256))
    frame_resized = frame_resized / 255.0
    frame_resized = frame_resized.astype(np.float32)
    frame_resized = np.expand_dims(frame_resized, axis=0)

    # Прогон через модель
    pred_mask = model.predict(frame_resized)[0]
    
    pred_mask = cv2.resize(pred_mask, (w, h))
    pred_mask = (pred_mask > 0.5).astype(np.uint8)

    # Выделение человека
    person = cv2.bitwise_and(original_frame, original_frame, mask=pred_mask)

    return person

In [19]:
def process_frame_combined(frame, it, model1, model2):
    """Обрабатывает один кадр, выделяя человека и удаляя фон."""
    original_frame = frame.copy()
    h, w, _ = frame.shape

    # Подготовка кадра для модели
    frame_resized = cv2.resize(frame, (256, 256))
    frame_resized = frame_resized / 255.0
    frame_resized = frame_resized.astype(np.float32)
    frame_resized = np.expand_dims(frame_resized, axis=0)

    # Прогон через модель
    if it % 2 == 0:
        pred_mask = model1.predict(frame_resized)[0]
    else:
        pred_mask = model2.predict(frame_resized)[0]
    
    pred_mask = cv2.resize(pred_mask, (w, h))
    pred_mask = (pred_mask > 0.5).astype(np.uint8)

    # Выделение человека
    person = cv2.bitwise_and(original_frame, original_frame, mask=pred_mask)

    return person

In [30]:
def process_video(input_path, output_path, model1, model2):
    """Обрабатывает видео, удаляя фон на каждом кадре."""
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    if not cap.isOpened():
        print("Ошибка при открытии видео!")
        return

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Инициализация сохранения выходного видео
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    it = 0
    for _ in tqdm(range(frame_count), desc="Обработка кадров"):
        ret, frame = cap.read()
        if not ret:
            break

        # Обработка кадра
        if model2 == '':
            person_frame = process_frame_one_model(frame, it, model1)
        else:
            person_frame = process_frame_combined(frame, it, model1, model2)
            
        out.write(person_frame)
        it+=1

    cap.release()
    out.release()
    print("Обработка завершена, результат сохранен:", output_path)


In [21]:
input_video = "/kaggle/input/videoformodel/videoShort.mp4"

In [23]:
output_video_segnet = "output_video_segnet.mp4"
output_video_unet = "output_video_unet.mp4"
output_video_combined = "output_videot_combined.mp4"

In [31]:
process_video(input_video, output_video_segnet, segnet_model, '')

Обработка кадров:   0%|          | 0/979 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 774ms/step


Обработка кадров:   0%|          | 1/979 [00:00<13:49,  1.18it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Обработка кадров:   0%|          | 3/979 [00:01<04:31,  3.59it/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Обработка кадров: 100%|█████████▉| 977/979 [01:19<00:00, 12.53it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Обработка кадров: 100%|██████████| 979/979 [01:19<00:00, 12.32it/s]

Обработка завершена, результат сохранен: output_video_segnet.mp4


In [33]:
process_video(input_video, output_video_unet, unet_model, '')

Обработка кадров:   0%|          | 0/979 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 893ms/step


Обработка кадров:   0%|          | 1/979 [00:00<15:39,  1.04it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


Обработка кадров:   0%|          | 3/979 [00:01<05:01,  3.24it/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


Обработка кадров: 100%|█████████▉| 977/979 [01:23<00:00, 11.91it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Обработка кадров: 100%|██████████| 979/979 [01:23<00:00, 11.71it/s]

Обработка завершена, результат сохранен: output_video_segnet_unet.mp4


In [34]:
process_video(input_video, output_video_combined, segnet_model, unet_model)

Обработка кадров:   0%|          | 0/979 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Обработка кадров:   0%|          | 2/979 [00:00<01:23, 11.68it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


Обработка кадров:   0%|          | 4/979 [00:00<01:23, 11.68it/s]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


Обработка кадров: 100%|█████████▉| 978/979 [01:20<00:00, 12.31it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


Обработка кадров: 100%|██████████| 979/979 [01:20<00:00, 12.11it/s]

Обработка завершена, результат сохранен: output_video_segnet_combined.mp4
